In [1]:
import numpy as np
import os
from pathlib import Path
from alphabet import Alphabet

In [25]:
alefs = [ "ا", "أ", "آ", "إ" ]

In [26]:
chars = ' ABCDEFGHIJKLMNOPQRSTUVWXYZÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤ0123456789!"%(),./:=?[]«»،؛؟-+*'

In [27]:
letters = {}
letters_positions={}
with open('letters.csv','r', encoding='utf-8') as inf:
    for line in inf:
        key,val=line.split(',')
        letters[key]=val.strip()
letters[',']=','
with open('letters_positions.csv','r', encoding='utf-8') as inf:
    for idx,line in enumerate(inf):       
        letters_positions[line.strip()]=idx

In [28]:
def is_two_pos(txt):
    return txt in  [ "أ", "ا", "إ", "آ", "ؤ", "د", "ذ", "ر", "ز", "و", "ة", "ء", "لا", "لأ", "لآ", "لإ","ا" ]

In [29]:
def is_isoleted(txt):
    return txt in ['0','1','2','3','4','5','6','7','8','9','!','"','%','(',')',',','.','/',':','=','?','[',']','«','»','،','؛','؟','–','+',' ','ء','.',',',':',' ']

In [30]:
def get_letter_position( cur,  pre =None, nex = None):
    if pre == None and nex==None:
        return ''
    if is_isoleted(cur):
        return ''
    if is_two_pos(cur):
        if is_two_pos(pre) or pre == None or is_isoleted(pre):
            return ''
        else:
            return "_E"
    else:
        if pre != None and nex!=None and not is_two_pos(pre) and not is_isoleted(nex) and not is_isoleted(pre):
            return '_M'
        elif (pre == None or is_isoleted(pre)   or is_two_pos(pre)  ) and (nex != None and not is_isoleted(nex)  ):
            return '_S'
        elif (nex ==None or  is_isoleted(nex) ) and not (is_two_pos(pre) or is_isoleted(pre) ):
            return '_E'
    return ''

In [2]:
text='« =  امن بة كثير من الرجال والنساء والغلام'

In [32]:
def text2label(text): 
    encoded=''
    pre=None
    nex=None
    cont=False
    if len(text)==1:
        encoded=chars[letters_positions[letters[text]]]
        return encoded
    for idx,cur in enumerate(text):    
        if cont:
            cont=False
            continue
        if(len(text)==1):
            print(letters_positions[letters[text]])
        if idx == 0 :
            pre=None
            nex=text[idx+1]
        elif idx==len(text)-1:
            pre=text[idx-1]
            nex=None
        else:
            pre=text[idx-1]
            nex=text[idx+1]
            
        if cur=='ل' and nex in alefs:
            cur=cur+nex
            cont=True
            if idx+1==len(text)-1:
                nex=None
            else:
                nex=text[idx+2]
        encoded=encoded+chars[letters_positions[letters[cur] + get_letter_position(cur, pre, nex)]]
    return encoded
    #     print('pre:'+str(pre)+',cur:'+cur+',nex:'+str(nex))

In [33]:
text2label(text)


' NĈĎ QU ĀÁěÔ ĈĎ NĄÔÄOă ēNĄčÙOA ēNĄõĞć'

In [34]:
import pandas as pd 

In [35]:
df=pd.read_csv('labels.csv')


In [36]:
df['labels']=df['text'].apply(lambda x:text2label(x))

In [37]:
df.head()

,file_name,text,labels
0,1.txt,ذهب احمد الى المدرسة,ÑĐS NÈĉÐ NĄĘ NĄĉÐÓØU
1,2.txt,الانسان,NĝČÙOċ
2,3.txt,اللغة العربية,NĄąõU NĄñÔQěU


In [38]:
# create target inputs
#write labels to files to be placed as pairs with feature file e.g  1_input.npy  as feature and 1_target.npy as label
alphabet=Alphabet('alphabet.txt')
for idx,row in df.iterrows():
    print(alphabet.encode(row['labels']))
    np.save('data/'+row['file_name'].replace('.txt','_target.npy'),alphabet.encode(row['labels']))

[44, 107, 19, 0, 14, 35, 100, 43, 0, 14, 95, 115, 0, 14, 95, 100, 43, 46, 51, 21]
[14, 120, 103, 52, 15, 102]
[14, 95, 96, 80, 21, 0, 14, 95, 76, 47, 17, 118, 21]


In [4]:
!pwd

/media/khalil/D/projects/DeepOnKHATT/data_utils/features


In [ ]:
from extract_features import process_single_file
from lang_trans.arabic import buckwalter

In [48]:
train_data_path = "../OnlineKHATTData/Training/"
train_save = "onlineKhatt/train"
val_data_path = "../OnlineKHATTData/Validation/"
dev_save = "onlineKhatt/dev"
test_save = "onlineKhatt/test"

In [ ]:
backwalter_labels = ["'", "|", ">", "&", "<", "}", "A", "b", "p", "t", "v", "j", "H", "x", "d", "*",
                  "r", "z", "s", "$", "S", "D", "T", "Z", "E", "g", "_", "f", "q", "k",  "l", "m", 
                    "n", "h", "w", "Y", "y", "F", "N", "K", "a", "u", "i",  "~", "o", "`", "{", "P",
                    "J", "V", "G", "#", " "]
with open("backwalter_labels.txt","w") as f:
    f.writelines("\n".join(backwalter_labels))

In [ ]:
# create data input
ink_paths = list(Path(train_data_path).glob("*_coor.txt"))
save_dir = Path(train_save)
os.makedirs(save_dir,exist_ok=True)
print(len(ink_paths))
for filename in ink_paths :
        with open(filename) as file:
            process_single_file(file,filename, f'{save_dir}/{filename.stem}_input.npy', normalize=True)
            file.close()
            try:
                os.rename(filename,'processed/'+filename)
            except Exception as e:
                pass

In [15]:
# create target inputs
alphabet=Alphabet('backwalter_labels.txt')
text_target_path = list(Path(train_data_path).glob("*.txt"))
save_dir = Path(train_save)
os.makedirs(save_dir,exist_ok=True)
for file_name in text_target_path:
    # print(alphabet.encode(row['labels']))
    # np.save(str(save_dir)+'/'+file_name.replace('.txt','_target.npy'), alphabet.encode(row['labels']))
    unicode = ['\ufeff','\xa0','،','–','‐','؛',',','"','؟','\n','‘','/','%']
    if "coor" not in file_name.name:
        with open(file_name,'r') as f:
            text = f.read()
            for i in unicode:
                if i in text:
                    text= text.replace(i,' ')
        label = buckwalter.transliterate(text.strip())
        np.save(str(save_dir)+'/'+ file_name.name.replace('.txt','_target.npy'), alphabet.encode(label))

In [ ]:
# create data input
ink_paths = list(Path(val_data_path).glob("*_coor.txt"))
save_dir = Path(val_save)
os.makedirs(save_dir,exist_ok=True)
print(len(ink_paths))
for filename in ink_paths :
        with open(filename) as file:
            process_single_file(file,filename, f'{save_dir}/{filename.stem}_input.npy', normalize=True)
            file.close()
            try:
                os.rename(filename,'processed/'+filename)
            except Exception as e:
                pass

# create target inputs
alphabet=Alphabet('backwalter_labels.txt')
text_target_path = list(Path(val_data_path).glob("*.txt"))
save_dir = Path(val_save)
os.makedirs(save_dir,exist_ok=True)
for file_name in text_target_path:
    # print(alphabet.encode(row['labels']))
    # np.save(str(save_dir)+'/'+file_name.replace('.txt','_target.npy'), alphabet.encode(row['labels']))
    unicode = ['\ufeff','\xa0','،','–','‐','؛',',','"','؟','\n','‘']
    if "coor" not in file_name.name:
        with open(file_name,'r') as f:
            text = f.read()
            for i in unicode:
                if i in text:
                    text= text.replace(i,' ')
        label = buckwalter.transliterate(text.strip())
        np.save(str(save_dir)+'/'+ file_name.name.replace('.txt','_target.npy'), alphabet.encode(label))

In [4]:
from random import shuffle

In [32]:
val_files = os.listdir(val_save)
print(len(val_files))
val_files_names = [f.rsplit('_',1)[0] for f in val_files if "coor" not in f]
shuffle(val_files_names)
partition = len(val_files_names) // 2 + 1
val_files, test_files = val_files_names[:partition], val_files_names[partition:]

3066


In [37]:
# move test files under test dir
# change val to dev
import shutil
for file in test_files:
    shutil.move(os.path.join(val_save,file+"_input.npy"),os.path.join(test_save,file+"_input.npy"))
    shutil.move(os.path.join(val_save,file+"_target.npy"),os.path.join(test_save,file+"_target.npy"))


In [50]:
data_dir = test_save
for file in os.listdir(data_dir):
    if "_coor" in file:
        os.rename(os.path.join(data_dir,file), os.path.join(data_dir,file.replace('_coor','')))

In [53]:
buckwalter.transliterate(text)

"« = Amn bp kvyr mn AlrjAl wAlnsA' wAlglAm"

In [54]:
text

'« = امن بة كثير من الرجال والنساء والغلام'